Example 2-30

In [1]:
import pandas as pd
import numpy as np
import math

T = reservoir temperature, °F
pb = bubble-point pressure, psig
Bo = oil formation volume factor, bbl/STB
psep = separator pressure, psig
Tsep = separator temperature, °F
co = isothermal compressibility coefficient of the oil at a specified pressure, psi-1

In [2]:
#Oil # T pb Rs Bo ro co at p > pb psep Tsep API gg
data = [[1, 250, 2377, 751, 1.528, 38.13, 22.14 * 10-6, 2689, 150, 60, 47.1, 0.851],
        [2, 220, 2620, 768, 1.474, 40.95, 18.75 * 10-6, 2810, 100, 75, 40.7, 0.855],
        [3, 260, 2051, 693, 1.529, 37.37, 22.69 * 10-6, 2526, 100, 72, 48.6, 0.911],
        [4, 237, 2884, 968, 1.619, 38.92, 21.51 * 10-6, 2942, 60, 120, 40.5, 0.898],
        [5, 218, 3065, 943, 1.570, 37.70, 24.16 * 10-6, 3273, 200, 60, 44.2, 0.781],
        [6, 180, 4239, 807, 1.385, 46.79, 11.65 * 10-6, 4370, 85, 173, 27.3, 0.848]]
PVTData = pd.DataFrame(data, columns = ['Oil#', 'T', 'pb', 'Rs', 'Bo', 'ro', 'co', 'p > pb', 'Psep', 'Tsep', 'API', 'Yg'])
PVTData

,Oil#,T,pb,Rs,Bo,ro,co,p > pb,Psep,Tsep,API,Yg
0,1,250,2377,751,1.528,38.13,215.4,2689,150,60,47.1,0.851
1,2,220,2620,768,1.474,40.95,181.5,2810,100,75,40.7,0.855
2,3,260,2051,693,1.529,37.37,220.9,2526,100,72,48.6,0.911
3,4,237,2884,968,1.619,38.92,209.1,2942,60,120,40.5,0.898
4,5,218,3065,943,1.570,37.70,235.6,3273,200,60,44.2,0.781
5,6,180,4239,807,1.385,46.79,110.5,4370,85,173,27.3,0.848


In [3]:
for well in PVTData['Oil#']:
    Yo = 141.5 / (131.5 + PVTData['API'])
    BoStanding = 0.9759 + 0.000120 * (PVTData['Rs']*(PVTData['Yg']/Yo)**0.5+1.25*(PVTData['T']-460)) ** 1.2 
BoStanding

0    1.191772
1    1.175586
2    1.181962
3    1.304765
4    1.248285
5    1.152939
dtype: float64

In [4]:
#Example 2-17: Standing’s Correlation
for well in PVTData['Oil#']:
    x = 0.0125 * PVTData['API'] - 0.00091 * (PVTData['T'])
    x10 = 10**x
    RsStanding = PVTData['Yg']*((PVTData['pb']/18.2+1.4)*10**round(x,3))**1.2048
    ErrorStanding = (RsStanding / PVTData['Rs']-1) * 100
output = pd.DataFrame(list(zip(x, x10, RsStanding, ErrorStanding)), columns = ['x', '10x', 'RsStanding', 'Error']) 
output

,x,10x,RsStanding,Error
0,0.36125,2.297471,831.279855,10.689728
1,0.30855,2.034932,811.986310,5.727384
2,0.37090,2.349092,767.505763,10.751192
3,0.29058,1.952450,909.795729,-6.012838
4,0.35412,2.260060,1013.443345,7.470132
5,0.17745,1.504700,992.599608,22.998712


In [46]:
#Example 2-18: The Vasquez-Beggs Correlation
Coefficient = [['C1', 0.0362, 0.0178], ['C2', 1.0937, 1.1870], ['C3', 25.7240, 23.931]]
Ygs = []
for well in PVTData['Oil#']:
#PVTData['Psep']  
    Ygs = PVTData['Yg'] * (1 + 5.912 * 10**(-5) * PVTData['API'] * PVTData['T'] * math.log((PVTData['Psep'][well-1]+1)/114.7,10))
    if PVTData['API'][well-1] < 30:
        C1 = Coefficient[0][1]
        C2 = Coefficient[1][1]
        C3 = Coefficient[2][1]
    else:
        C1 = Coefficient[0][2]
        C2 = Coefficient[1][2]
        C3 = Coefficient[2][2]
    RsVasquez = C1 * Ygs *  PVTData['Psep'] ** C2 * math.exp(C3 * PVTData['API'][well-1]/PVTData['T'][well-1])
    ErrorVasquez = (RsVasquez / PVTData['Rs']-1) * 100
output = pd.DataFrame(list(zip(Ygs, RsVasquez, ErrorVasquez)), columns = ['Ygs', 'RsVasquez', 'Error']) 
output

,Ygs,RsVasquez,Error
0,0.776910,333.778638,-55.555441
1,0.798395,220.148041,-71.334891
2,0.825887,227.728463,-67.138750
3,0.834269,131.573101,-86.407738
4,0.725358,426.860570,-54.733768
5,0.817189,188.636217,-76.625004


In [45]:
# Example 2-19: Glaso’s Correlation
for well in PVTData['Oil#']:
    x = 2.8869-(14.1811-3.3093*math.log(PVTData['Psep'][well-1], 10))**0.5
    Pb = 10 ** x
    RsGlaso = PVTData['Yg']*((PVTData['API']**0.989/PVTData['T']**0.172)*Pb)**1.2255
RsGlaso

0    37.011232
1    32.003555
2    40.816339
3    32.892862
4    32.370039
5    20.407759
dtype: float64

In [48]:
# Example 2-20: Marhoun’s Correlation
#Yg = gas specific gravity
#Yo = stock-tank oil gravity
#T = temperature, °R
#a–e = coefficients of the above equation having these values: 
a = 185.843208
b = 1.877840
c = -3.1437
d = -1.32657
e = 1.398441
for well in PVTData['Oil#']:
    Yo = 141.5 / (131.5 + PVTData['API'])
    RsMarhoun = (a * (PVTData['Yg']**b) * (Yo**c) * (PVTData['T']**d) * PVTData['Psep'])**e
RsMarhoun

0    106.777134
1     66.205091
2     69.870854
3     31.946521
4    152.895938
5     52.458322
dtype: float64

In [ ]:
# Example 2-21: Petrosky-Farshad Correlation
